In [85]:
from preprocess_mri import *

In [120]:
n3_dir = os.path.join('data', 'mri', 'ADNI', '136_S_1227', 'MPR____N3__Scaled')
files = find_nifti_files(n3_dir)

mni_path = os.path.join('mni', 'icbm152.nii')
mni_img = nil.image.load_img(mni_path)

aal = nil.datasets.fetch_atlas_aal(version='3v2')
atlas_img = nil.image.load_img(aal.maps)

for file in files:
    registered_img = register_to_mni(file, mni_img)
    segmented = segment_with_atlas(registered_img, atlas_img, aal.labels)
    print(f'Processed {file}: Registered shape {registered_img.shape}, Segmented shape {segmented.shape}')

[fetch_atlas_aal] Dataset found in C:\Users\Dhruv Khanna\nilearn_data\aal_3v2
Processed data\mri\ADNI\136_S_1227\MPR____N3__Scaled\2007-02-21_13_12_42.0\I66824\ADNI_136_S_1227_MR_MPR____N3__Scaled_Br_20070810000731580_S26837_I66824.nii: Registered shape (121, 145, 121), Segmented shape (166,)
Processed data\mri\ADNI\136_S_1227\MPR____N3__Scaled\2009-03-23_11_59_29.0\I143856\ADNI_136_S_1227_MR_MPR____N3__Scaled_Br_20090514132202278_S64957_I143856.nii: Registered shape (121, 145, 121), Segmented shape (166,)
